#Get going on sorting people into leagues for curling
KL 8/17/2016

This version randomizes the people first

In [1]:
if True:
    %reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import pandas as pd
import numpy as np
import math
import csv
import os.path

#used this to step into the function and debug it, also need line with Tracer()() 
from IPython.core.debugger import Tracer 

In [3]:
#read in the file with the information on people's priorities
#goal will be to use the version that comes out of the website
fName = 'TestPriority4.csv'
regInfo=pd.read_csv(fName)

Define some functions up top

In [4]:
def addOne(leagueDict,oneLeague,oneName):
    if len(leagueDict[oneLeague])==0: #nothing there yet
        leagueDict[oneLeague] = oneName
    else:     #need to append
        leagueDict[oneLeague] = leagueDict[oneLeague].append(oneName)
    return leagueDict

In [5]:
def addManager(r,manager):
    useRow = r.index[0]
    oneName = regSorted.loc[useRow,['lastname', 'firstname']] 
    oneName = oneName.to_frame().transpose() #need this format to make nice output
    leagueDict_open[manager] = oneName
    regSorted.loc[useRow,manager]=np.nan
    regTracking.loc[useRow,manager]='yes'

In [6]:
def doStuff(idx,regSorted,regTracking,leagueDict_open):
    #person is interested in open leagues, oneLeague will be index in their next choice
    oneLeague = regSorted.loc[idx,openLeagues].dropna().idxmin()

    oneName = regSorted.loc[idx,['lastname', 'firstname']] 
    oneName = oneName.to_frame().transpose() #need this format to make nice output

    if len(leagueDict_open[oneLeague]) < (nLimit.loc[oneLeague,'nLimit']): #room in league
        leagueDict_open = addOne(leagueDict_open,oneLeague,oneName)
        #now that I have added the person to a league, change their priority to NaN
        regSorted.loc[idx,oneLeague] = np.nan
        regTracking.loc[idx,oneLeague] = 'yes'

    elif len(leagueDict_open[oneLeague]) == (nLimit.loc[oneLeague,'nLimit']):
        #league is full...does this person have another league listed?
        #first, set current to NaN
        regSorted.loc[idx,oneLeague] = np.nan
        regTracking.loc[idx,oneLeague] = 'no' #did not get in
        nextPriority = regSorted.loc[idx,openLeagues].dropna().min()
        if math.isnan(nextPriority)!=1: #have a number
            #go to the next league on their list
            nextLeague = regSorted.loc[idx,openLeagues].dropna().idxmin()
            #only add person is league has space
            if len(leagueDict[nextLeague]) < (nLimit.loc[nextLeague,'nLimit']):
                leagueDict_open = addOne(leagueDict_open,nextLeague,oneName)
                regSorted.loc[idx,nextLeague] = np.nan
                regTracking.loc[idx,nextLeague] = 'yes'
            else:
                regTracking.loc[idx,nextLeague]='no'
    return regSorted, regTracking, leagueDict_open

get started on the actual lists

In [7]:
##OK, now that I have the data in Python, add some columns that I will need
#first, how many numbers so I need?
nPeople = len(regInfo.index)
regInfo['randomNumCol'] = np.random.choice(range(nPeople), nPeople,replace = False)
##so now, moving back to the list of people, sort the list by the random number
regSorted = regInfo.sort_values(by = 'randomNumCol', ascending=True)
regSorted = regSorted.set_index(['randomNumCol'])

#copy so I can have an output tracking what was done, make after I put in league managers
regTracking = regSorted.copy(deep=True)

In [8]:
#now, make up a series of variables that are the names of the leagues, 
#can pull the leageu names from imported CSV file. They won't move around,
#so just index in, but don't need the Men's league in here
openLeagues = list(regInfo)[5:12]
openLeagues.remove('League_4_Men')

In [9]:
openLeagues

['League_1_Sunday_Open',
 'League_2_Monday_Open',
 'League_3_Monday_Night_Open',
 'League_Tuesday_Open',
 'League_5_Wednesday_Open',
 'League_6_Thursday_Open']

In [10]:
#setup the dictionary to hold the people information (start with openLeagues only)
forDictionary = openLeagues
#forDictionary.extend(otherLeagues)
leagueDict_open = {}
for name in forDictionary:
    leagueDict_open[name] = pd.DataFrame()

In [11]:
#set the limit for each league:
sevenTeams = 27 #change to 28/27 (? should league manager goes in first)
fourteenTeams = 55

d = {'League_1_Sunday_Open': sevenTeams,
     'League_2_Monday_Open': sevenTeams,
     'League_3_Monday_Night_Open': fourteenTeams,
     'League_Tuesday_Open': sevenTeams,
     'League_5_Wednesday_Open': sevenTeams,
     'League_6_Thursday_Open': sevenTeams,}

nLimit = pd.DataFrame({'nLimit' : d})
del d

In [12]:
#put the league managers into their league and set their the choice for that league to NaN
manager = 'League_1_Sunday_Open'
r = regSorted.loc[(regSorted['lastname']=='Fragaszy') & (regSorted['firstname']=='Bill')]
addManager(r,manager)

manager = 'League_2_Monday_Open'
r = regSorted.loc[(regSorted['lastname']=='Brown') & (regSorted['firstname']=='Peter')]
addManager(r,manager)

manager = 'League_3_Monday_Night_Open'
r = regSorted.loc[(regSorted['lastname']=='DiMassa') & (regSorted['firstname']=='Diane')]
addManager(r,manager)

manager = 'League_Tuesday_Open'
r = regSorted.loc[(regSorted['lastname']=='Thomson') & (regSorted['firstname']=='Kenny')]
addManager(r,manager)

manager = 'League_5_Wednesday_Open'
r = regSorted.loc[(regSorted['lastname']=='Murphy') & (regSorted['firstname']=='Katie')]
addManager(r,manager)

manager = 'League_6_Thursday_Open'
r = regSorted.loc[(regSorted['lastname']=='Gallagher') & (regSorted['firstname']=='Bill')]
addManager(r,manager)

In [13]:
#go forwards through the list to start
for idx in (range(0,nPeople)):
    #only do something if a person is still looking for more leagues...
    rt = regTracking.loc[idx,openLeagues]
    cHave = len(rt[rt=='yes'])
    del rt
    if cHave < regTracking.loc[idx,'numberLeagues']: #person wants more 
        onePriority = regSorted.loc[idx,openLeagues].dropna().min() #any more things ranked?   
        if math.isnan(float(onePriority))==False:
            regSorted,regTracking,leagueDict = doStuff(idx,regSorted,regTracking,leagueDict_open)

In [14]:
#now go backwards through the list to fill the other spots
for idx in reversed(range(0,nPeople)):
    #only do something if a person is still looking for more leagues...
    rt = regTracking.loc[idx,openLeagues]
    cHave = len(rt[rt=='yes'])
    del rt
    if cHave < regTracking.loc[idx,'numberLeagues']: #person wants more 
        onePriority = regSorted.loc[idx,openLeagues].dropna().min() #any more things ranked?   
        if math.isnan(float(onePriority))==False:
            regSorted,regTracking,leagueDict = doStuff(idx,regSorted,regTracking,leagueDict_open)
        

In [15]:
openLeagues

['League_1_Sunday_Open',
 'League_2_Monday_Open',
 'League_3_Monday_Night_Open',
 'League_Tuesday_Open',
 'League_5_Wednesday_Open',
 'League_6_Thursday_Open']

In [16]:
#go forwards again...stop here...will be max 3, 4 pm leagues
for idx in (range(0,nPeople)):
    #only do something if a person is still looking for more leagues...
    rt = regTracking.loc[idx,openLeagues]
    cHave = len(rt[rt=='yes'])
    del rt
    if cHave < regTracking.loc[idx,'numberLeagues']: #person wants more 
        onePriority = regSorted.loc[idx,openLeagues].dropna().min() #any more things ranked?   
        if math.isnan(float(onePriority))==False:
            regSorted,regTracking,leagueDict = doStuff(idx,regSorted,regTracking,leagueDict_open)

In [17]:
#make a new dictionary for other leagues...concat with openLeagues when done
# do doubles and competitive league by hand...
otherLeagues = ['League_4_Men', 'League_7_Ladies' ,'League_Scrod']

leagueDict_other = {}
for name in otherLeagues:
    leagueDict_other[name] = pd.DataFrame()

In [18]:
#only have to go through once
for idx in range(0,nPeople):
    for oL in otherLeagues:
    
        oneName = regSorted.loc[idx,['lastname', 'firstname']] 
        oneName = oneName.to_frame().transpose() #need this format to make nice output
        if regSorted.loc[idx,oL]=='Yes': #careful with trailing spaces
            leagueDict_other = addOne(leagueDict_other,oL,oneName)
            #now that I have added the person to a league, change their priority to NaN
            regTracking.loc[idx,oL] = 'done'
            regSorted.loc[idx,oL] = np.nan

In [19]:
#combine the two dictionaries
leagueDict_all = leagueDict_open.copy()
leagueDict_all.update(leagueDict_other)

#combine the list of names
allLeagues = list(openLeagues)
allLeagues.extend(otherLeagues)

In [20]:
#finally, export the results (CAREFUL...will delete existing files)
for fn in allLeagues:
    fName = '%s.csv' % fn
    #check if the file exists...delete if it does
    if os.path.isfile(fName):
        os.remove(fName)
        
    leagueDict_all[fn].to_csv(fName)

In [21]:
if True:
    #now...print up a list of  problems...use information in regTracking
    #essentially..how many people have # 'yes' < numberLeagues
    for idx in range(0,nPeople):
        test = regTracking.loc[idx,openLeagues]
        
        oneName = regSorted.loc[idx,['lastname', 'firstname']] 
        oneName = oneName.to_frame().transpose() #need this format to make nice output
       
        if test.isnull().sum()!=6: #if al 6 are NaN, no open leagues
            testT = regTracking.loc[idx,openLeagues]
            nNo = testT[testT=='no']
            nYes = testT[testT=='yes']
            if (len(nYes)==0) & (len(nNo) > 0):
                print oneName['firstname'], oneName['lastname'], ' : got no open leagues'      

In [22]:
# leave here bc I keep having to look this up
#df.loc[df['column_name'] == some_value]
regTracking.loc[regTracking['lastname']=='Longnecker']

,Register Date,Event Name,firstname,lastname,numberLeagues,League_1_Sunday_Open,League_2_Monday_Open,League_3_Monday_Night_Open,League_Tuesday_Open,League_4_Men,...,League_6_Thursday_Open,League_7_Ladies,League_Friday_Doubles,League_Friday_Doubles2,League_Friday_Doubles3,Competitive_League,Competitive_League_Part2,League_Scrod,League_Day_Curling,League_Sub
randomNumCol,,,,,,,,,,,,,,,,,,,,,
141,12/30/2015 11:09,2015-2016 Second Half League Registration,Krista,Longnecker,1,NaN,NaN,yes,NaN,NaN,...,NaN,NaN,No,NaN,NaN,On a team,Longnecker,NaN,No,NaN


In [23]:
#regTracking.to_csv('temp7.csv')